In [ ]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 245kB 11.1MB/s 
     |████████████████████████████████| 122kB 22.4MB/s 
     |████████████████████████████████| 245kB 21.7MB/s 
     |████████████████████████████████| 2.3MB 15.5MB/s 
     |████████████████████████████████| 3.3MB 59.9MB/s 
     |████████████████████████████████| 901kB 48.2MB/s 
  Found existing installation: huggingface-hub 0.0.10
    Uninstalling huggingface-hub-0.0.10:
      Successfully uninstalled huggingface-hub-0.0.10


In [ ]:
import transformers
import datasets
import torch
from datasets import load_dataset

class NLITRReader(torch.utils.data.Dataset):
  def __init__(self, dataset_name, split_name, max_example_num=-1):
    self.dataset = load_dataset('nli_tr', dataset_name)
    self.split_name = split_name
    self.max_example_num = max_example_num

  def read(self):
      count = 0
      for example in self.dataset[self.split_name]:
          if example['label'] == -1: # skip examples having no gold value.
              continue
          count += 1
          if self.max_example_num > 0 and count >= self.max_example_num:
             break
          yield example

In [ ]:
import torch
class NLITRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
multinli_tr_dataset = load_dataset('nli_tr', 'multinli_tr')

Dataset nli_tr downloaded and prepared to /root/.cache/huggingface/datasets/nli_tr/multinli_tr/1.0.0/c2ddd0c0a70caddac6a81c2dae5ca7939f00060d517d08f1983927818dba6521. Subsequent calls will reuse this data.


In [ ]:
print(multinli_tr_dataset.shape)

{'train': (392702, 4), 'validation_matched': (10000, 4), 'validation_mismatched': (10000, 4)}


In [ ]:
train_data = multinli_tr_dataset['train']
valid_matched_data = multinli_tr_dataset['validation_matched']
valid_mismatched_data = multinli_tr_dataset['validation_mismatched']

In [ ]:
import pandas as pd
train_data_df = pd.DataFrame(train_data)

In [ ]:
train_data_df

,hypothesis,idx,label,premise
0,Ürün ve coğrafya krem kaymağını işe yarıyor.,0,1,Kavramsal olarak krem kaymağının iki temel boy...
1,"Eğer insanlar hatırlarsa, bir sonraki seviyeye...",1,0,Mevsim boyunca ve sanırım senin seviyendeyken ...
2,Ekibimin bir üyesi emirlerinizi büyük bir hass...,2,0,Numaramızdan biri talimatlarınızı birazdan yer...
3,Bu bilgi onlara ait.,3,0,Bunu nereden biliyorsun? Bütün bunlar yine onl...
4,Tenis ayakkabılarının çeşitli fiyatları var.,4,1,"Evet, bak ne diyeceğim. Eğer tenis ayakkabılar..."
...,...,...,...,...
392697,California daha iyisini yapamaz.,392697,2,"Açıkçası, Kaliforniya daha iyisini yapabilir v..."
392698,Orijinal binaların çoğu marketler tarafından d...,392698,1,Bir zamanlar Avrupa'nın en güzel caddesi olara...
392699,İngiliz Raj hala güçlenirken yüzen evlerin gel...,392699,0,"Evdeki tekneler, İngiliz Raj'ının doruk gününü..."
392700,Ölüm ilanları güzel ve eğlence endüstrisindeki...,392700,1,"Ölüm ilanları, kendi adını taşıyan sendikasyon..."


In [ ]:
train_data_df['hypothesis'][2]

'Ekibimin bir üyesi emirlerinizi büyük bir hassasiyetle yerine getirecektir.'

In [ ]:
train_data_df['premise'][2]

'Numaramızdan biri talimatlarınızı birazdan yerine getirecektir.'

In [ ]:
del train_data_df['idx']
train_data_df

,hypothesis,label,premise
0,Ürün ve coğrafya krem kaymağını işe yarıyor.,1,Kavramsal olarak krem kaymağının iki temel boy...
1,"Eğer insanlar hatırlarsa, bir sonraki seviyeye...",0,Mevsim boyunca ve sanırım senin seviyendeyken ...
2,Ekibimin bir üyesi emirlerinizi büyük bir hass...,0,Numaramızdan biri talimatlarınızı birazdan yer...
3,Bu bilgi onlara ait.,0,Bunu nereden biliyorsun? Bütün bunlar yine onl...
4,Tenis ayakkabılarının çeşitli fiyatları var.,1,"Evet, bak ne diyeceğim. Eğer tenis ayakkabılar..."
...,...,...,...
392697,California daha iyisini yapamaz.,2,"Açıkçası, Kaliforniya daha iyisini yapabilir v..."
392698,Orijinal binaların çoğu marketler tarafından d...,1,Bir zamanlar Avrupa'nın en güzel caddesi olara...
392699,İngiliz Raj hala güçlenirken yüzen evlerin gel...,0,"Evdeki tekneler, İngiliz Raj'ının doruk gününü..."
392700,Ölüm ilanları güzel ve eğlence endüstrisindeki...,1,"Ölüm ilanları, kendi adını taşıyan sendikasyon..."


In [ ]:
columns_titles = ["label","premise","hypothesis"]

df_reorder=train_data_df.reindex(columns=columns_titles)
df_reorder

,label,premise,hypothesis
0,1,Kavramsal olarak krem kaymağının iki temel boy...,Ürün ve coğrafya krem kaymağını işe yarıyor.
1,0,Mevsim boyunca ve sanırım senin seviyendeyken ...,"Eğer insanlar hatırlarsa, bir sonraki seviyeye..."
2,0,Numaramızdan biri talimatlarınızı birazdan yer...,Ekibimin bir üyesi emirlerinizi büyük bir hass...
3,0,Bunu nereden biliyorsun? Bütün bunlar yine onl...,Bu bilgi onlara ait.
4,1,"Evet, bak ne diyeceğim. Eğer tenis ayakkabılar...",Tenis ayakkabılarının çeşitli fiyatları var.
...,...,...,...
392697,2,"Açıkçası, Kaliforniya daha iyisini yapabilir v...",California daha iyisini yapamaz.
392698,1,Bir zamanlar Avrupa'nın en güzel caddesi olara...,Orijinal binaların çoğu marketler tarafından d...
392699,0,"Evdeki tekneler, İngiliz Raj'ının doruk gününü...",İngiliz Raj hala güçlenirken yüzen evlerin gel...
392700,1,"Ölüm ilanları, kendi adını taşıyan sendikasyon...",Ölüm ilanları güzel ve eğlence endüstrisindeki...


In [ ]:
df_reorder.columns = ['gold_label', 'sentence1', 'sentence2']
df_reorder

,gold_label,sentence1,sentence2
0,1,Kavramsal olarak krem kaymağının iki temel boy...,Ürün ve coğrafya krem kaymağını işe yarıyor.
1,0,Mevsim boyunca ve sanırım senin seviyendeyken ...,"Eğer insanlar hatırlarsa, bir sonraki seviyeye..."
2,0,Numaramızdan biri talimatlarınızı birazdan yer...,Ekibimin bir üyesi emirlerinizi büyük bir hass...
3,0,Bunu nereden biliyorsun? Bütün bunlar yine onl...,Bu bilgi onlara ait.
4,1,"Evet, bak ne diyeceğim. Eğer tenis ayakkabılar...",Tenis ayakkabılarının çeşitli fiyatları var.
...,...,...,...
392697,2,"Açıkçası, Kaliforniya daha iyisini yapabilir v...",California daha iyisini yapamaz.
392698,1,Bir zamanlar Avrupa'nın en güzel caddesi olara...,Orijinal binaların çoğu marketler tarafından d...
392699,0,"Evdeki tekneler, İngiliz Raj'ının doruk gününü...",İngiliz Raj hala güçlenirken yüzen evlerin gel...
392700,1,"Ölüm ilanları, kendi adını taşıyan sendikasyon...",Ölüm ilanları güzel ve eğlence endüstrisindeki...


In [ ]:
import os

In [ ]:
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/multinli_train.csv', sep='\t', index=False, header=True)

In [ ]:
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/multinli_train_index.csv', sep='\t', index=True, header=True)

In [ ]:
#VALIDATION MATCHED
import pandas as pd
valid_matched_data_df = pd.DataFrame(valid_matched_data)
valid_matched_data_df

,hypothesis,idx,label,premise
0,Herkes gerçekten en yeni faydaları seviyor,0,1,Yeni haklar yeterince güzel.
1,Web sitesinde yer alan Hükümet Yürütme makalel...,1,2,"Bu site, tüm ödül kazananların bir listesini v..."
2,"Çoğunlukla ondan hoşlanıyorum, ama yine de bir...",2,0,Bilemiyorum. Onunla ilgili karışık duygularım ...
3,En sevdiğim restoranlar her zaman evimden en a...,3,2,"Evet, sanırım en sevdiğim restoran her zaman e..."
4,Tam olarak biliyorum.,4,2,Bilmiyorum. Çok fazla kamp yapıyor musun?
...,...,...,...,...
9995,LSC uzun zamandır devlet dağıtım sistemlerini ...,9995,1,"1998 yılından bu yana LSC, daha güçlü ve etkil..."
9996,Amerika Birleşik Devletleri'nde çağrı cihazlar...,9996,2,Amerika'daki çağrı cihazlarının yüzde 80'i uza...
9997,FDA atölye çalışmaları yapmak üzere ayarlandı.,9997,0,"Son olarak, FDA atölye çalışmaları yürütecek, ..."
9998,Cirque du Soleil uluslararası bir topluluğu.,9998,0,Cirque du Soleil en beğenilen uluslararası top...


In [ ]:
del valid_matched_data_df['idx']
valid_matched_data_df

,hypothesis,label,premise
0,Herkes gerçekten en yeni faydaları seviyor,1,Yeni haklar yeterince güzel.
1,Web sitesinde yer alan Hükümet Yürütme makalel...,2,"Bu site, tüm ödül kazananların bir listesini v..."
2,"Çoğunlukla ondan hoşlanıyorum, ama yine de bir...",0,Bilemiyorum. Onunla ilgili karışık duygularım ...
3,En sevdiğim restoranlar her zaman evimden en a...,2,"Evet, sanırım en sevdiğim restoran her zaman e..."
4,Tam olarak biliyorum.,2,Bilmiyorum. Çok fazla kamp yapıyor musun?
...,...,...,...
9995,LSC uzun zamandır devlet dağıtım sistemlerini ...,1,"1998 yılından bu yana LSC, daha güçlü ve etkil..."
9996,Amerika Birleşik Devletleri'nde çağrı cihazlar...,2,Amerika'daki çağrı cihazlarının yüzde 80'i uza...
9997,FDA atölye çalışmaları yapmak üzere ayarlandı.,0,"Son olarak, FDA atölye çalışmaları yürütecek, ..."
9998,Cirque du Soleil uluslararası bir topluluğu.,0,Cirque du Soleil en beğenilen uluslararası top...


In [ ]:
columns_titles = ["label","premise","hypothesis"]

df_reorder_valid_matched=valid_matched_data_df.reindex(columns=columns_titles)
df_reorder_valid_matched

,label,premise,hypothesis
0,1,Yeni haklar yeterince güzel.,Herkes gerçekten en yeni faydaları seviyor
1,2,"Bu site, tüm ödül kazananların bir listesini v...",Web sitesinde yer alan Hükümet Yürütme makalel...
2,0,Bilemiyorum. Onunla ilgili karışık duygularım ...,"Çoğunlukla ondan hoşlanıyorum, ama yine de bir..."
3,2,"Evet, sanırım en sevdiğim restoran her zaman e...",En sevdiğim restoranlar her zaman evimden en a...
4,2,Bilmiyorum. Çok fazla kamp yapıyor musun?,Tam olarak biliyorum.
...,...,...,...
9995,1,"1998 yılından bu yana LSC, daha güçlü ve etkil...",LSC uzun zamandır devlet dağıtım sistemlerini ...
9996,2,Amerika'daki çağrı cihazlarının yüzde 80'i uza...,Amerika Birleşik Devletleri'nde çağrı cihazlar...
9997,0,"Son olarak, FDA atölye çalışmaları yürütecek, ...",FDA atölye çalışmaları yapmak üzere ayarlandı.
9998,0,Cirque du Soleil en beğenilen uluslararası top...,Cirque du Soleil uluslararası bir topluluğu.


In [ ]:
df_reorder_valid_matched.columns = ['gold_label', 'sentence1', 'sentence2']
df_reorder_valid_matched

,gold_label,sentence1,sentence2
0,1,Yeni haklar yeterince güzel.,Herkes gerçekten en yeni faydaları seviyor
1,2,"Bu site, tüm ödül kazananların bir listesini v...",Web sitesinde yer alan Hükümet Yürütme makalel...
2,0,Bilemiyorum. Onunla ilgili karışık duygularım ...,"Çoğunlukla ondan hoşlanıyorum, ama yine de bir..."
3,2,"Evet, sanırım en sevdiğim restoran her zaman e...",En sevdiğim restoranlar her zaman evimden en a...
4,2,Bilmiyorum. Çok fazla kamp yapıyor musun?,Tam olarak biliyorum.
...,...,...,...
9995,1,"1998 yılından bu yana LSC, daha güçlü ve etkil...",LSC uzun zamandır devlet dağıtım sistemlerini ...
9996,2,Amerika'daki çağrı cihazlarının yüzde 80'i uza...,Amerika Birleşik Devletleri'nde çağrı cihazlar...
9997,0,"Son olarak, FDA atölye çalışmaları yürütecek, ...",FDA atölye çalışmaları yapmak üzere ayarlandı.
9998,0,Cirque du Soleil en beğenilen uluslararası top...,Cirque du Soleil uluslararası bir topluluğu.


In [ ]:
pd.DataFrame(df_reorder_valid_matched).to_csv(os.getcwd()+'/valid_matched_noIndex.csv', sep='\t', index=False, header=True)


In [ ]:
pd.DataFrame(df_reorder_valid_matched).to_csv(os.getcwd()+'/valid_matched_index.csv', sep='\t', index=True, header=True)


In [ ]:
#VALIDATION MISMATCHED
valid_mismatched_data_df = pd.DataFrame(valid_mismatched_data)
valid_mismatched_data_df

,hypothesis,idx,label,premise
0,Katkılarınızın öğrencilerimizin eğitimine fayd...,0,2,"Katkınız, öğrencilerimize kaliteli bir eğitim ..."
1,Sözlükler gerçekten iki benzersiz ikame alıştı...,1,2,"Cevap onların nedeni ile ilgisi yoktur, ancak ..."
2,Biz Florentine Terine bir yemek servis.,2,0,Penis ve tavuk ciğeri ile yapılan Floransalı t...
3,Carl Newton ve ben seninle daha önce hiç temas...,3,2,Birkaç ay önce Carl Newton ve ben sizden India...
4,Dünya'da neden yaşadığımı henüz bilmiyorum.,4,0,Ben bu dünyadaydım. Bir sebepten dolayı bu dün...
...,...,...,...,...
9995,Görebiliyor musun?,9995,2,Bunu izliyor musun?
9996,"Batı kulağına göre, dil özelliklerinin en az ö...",9996,2,"Batı kulağına, dil özelliklerinin en öngörüleb..."
9997,Kayıt cihazı seslerin hiçbirini yakalamadı.,9997,2,"Kayıt cihazı, yüksek sesle çarpışmalar, çığlık..."
9998,"Bu konuda iyi hissediyorsun, değil mi?",9998,1,Bu iyi bir tutum!


In [ ]:
del valid_mismatched_data_df['idx']
valid_mismatched_data_df

,hypothesis,label,premise
0,Katkılarınızın öğrencilerimizin eğitimine fayd...,2,"Katkınız, öğrencilerimize kaliteli bir eğitim ..."
1,Sözlükler gerçekten iki benzersiz ikame alıştı...,2,"Cevap onların nedeni ile ilgisi yoktur, ancak ..."
2,Biz Florentine Terine bir yemek servis.,0,Penis ve tavuk ciğeri ile yapılan Floransalı t...
3,Carl Newton ve ben seninle daha önce hiç temas...,2,Birkaç ay önce Carl Newton ve ben sizden India...
4,Dünya'da neden yaşadığımı henüz bilmiyorum.,0,Ben bu dünyadaydım. Bir sebepten dolayı bu dün...
...,...,...,...
9995,Görebiliyor musun?,2,Bunu izliyor musun?
9996,"Batı kulağına göre, dil özelliklerinin en az ö...",2,"Batı kulağına, dil özelliklerinin en öngörüleb..."
9997,Kayıt cihazı seslerin hiçbirini yakalamadı.,2,"Kayıt cihazı, yüksek sesle çarpışmalar, çığlık..."
9998,"Bu konuda iyi hissediyorsun, değil mi?",1,Bu iyi bir tutum!


In [ ]:
columns_titles = ["label","premise","hypothesis"]

df_reorder_valid_mismatched=valid_mismatched_data_df.reindex(columns=columns_titles)
df_reorder_valid_mismatched

,label,premise,hypothesis
0,2,"Katkınız, öğrencilerimize kaliteli bir eğitim ...",Katkılarınızın öğrencilerimizin eğitimine fayd...
1,2,"Cevap onların nedeni ile ilgisi yoktur, ancak ...",Sözlükler gerçekten iki benzersiz ikame alıştı...
2,0,Penis ve tavuk ciğeri ile yapılan Floransalı t...,Biz Florentine Terine bir yemek servis.
3,2,Birkaç ay önce Carl Newton ve ben sizden India...,Carl Newton ve ben seninle daha önce hiç temas...
4,0,Ben bu dünyadaydım. Bir sebepten dolayı bu dün...,Dünya'da neden yaşadığımı henüz bilmiyorum.
...,...,...,...
9995,2,Bunu izliyor musun?,Görebiliyor musun?
9996,2,"Batı kulağına, dil özelliklerinin en öngörüleb...","Batı kulağına göre, dil özelliklerinin en az ö..."
9997,2,"Kayıt cihazı, yüksek sesle çarpışmalar, çığlık...",Kayıt cihazı seslerin hiçbirini yakalamadı.
9998,1,Bu iyi bir tutum!,"Bu konuda iyi hissediyorsun, değil mi?"


In [ ]:
df_reorder_valid_mismatched.columns = ['gold_label', 'sentence1', 'sentence2']
df_reorder_valid_mismatched

,gold_label,sentence1,sentence2
0,2,"Katkınız, öğrencilerimize kaliteli bir eğitim ...",Katkılarınızın öğrencilerimizin eğitimine fayd...
1,2,"Cevap onların nedeni ile ilgisi yoktur, ancak ...",Sözlükler gerçekten iki benzersiz ikame alıştı...
2,0,Penis ve tavuk ciğeri ile yapılan Floransalı t...,Biz Florentine Terine bir yemek servis.
3,2,Birkaç ay önce Carl Newton ve ben sizden India...,Carl Newton ve ben seninle daha önce hiç temas...
4,0,Ben bu dünyadaydım. Bir sebepten dolayı bu dün...,Dünya'da neden yaşadığımı henüz bilmiyorum.
...,...,...,...
9995,2,Bunu izliyor musun?,Görebiliyor musun?
9996,2,"Batı kulağına, dil özelliklerinin en öngörüleb...","Batı kulağına göre, dil özelliklerinin en az ö..."
9997,2,"Kayıt cihazı, yüksek sesle çarpışmalar, çığlık...",Kayıt cihazı seslerin hiçbirini yakalamadı.
9998,1,Bu iyi bir tutum!,"Bu konuda iyi hissediyorsun, değil mi?"


In [ ]:
pd.DataFrame(df_reorder_valid_mismatched).to_csv(os.getcwd()+'/valid_mismatched_noIndex.csv', sep='\t', index=False, header=True)

In [ ]:
pd.DataFrame(df_reorder_valid_mismatched).to_csv(os.getcwd()+'/valid_mismatched_index.csv', sep='\t', index=True, header=True)

In [ ]:
#TSV

In [ ]:
pd.DataFrame(df_reorder_valid_mismatched).to_csv(os.getcwd()+'/valid_mismatched_index.tsv', sep='\t', index=True, header=True)

In [ ]:
pd.DataFrame(df_reorder_valid_mismatched).to_csv(os.getcwd()+'/valid_mismatched_noIndex.tsv', sep='\t', index=False, header=True)

In [ ]:
pd.DataFrame(df_reorder_valid_matched).to_csv(os.getcwd()+'/valid_matched_noIndex.tsv', sep='\t', index=False, header=True)

In [ ]:
pd.DataFrame(df_reorder_valid_matched).to_csv(os.getcwd()+'/valid_matched_index.tsv', sep='\t', index=True, header=True)

In [ ]:
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/multinli_train_index.tsv', sep='\t', index=True, header=True)

In [ ]:
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/multinli_train_noIndex.tsv', sep='\t', index=False, header=True)